### Import libraries

In [1]:
import pandas
import numpy
import os
from dotenv import load_dotenv
import mysql.connector

### Database connection

In [ ]:
# Load variables from .env
load_dotenv()

def connect_to_db():
    conn = mysql.connector.connect(
        host=os.getenv("DB_HOST"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database=os.getenv("DB_NAME")
    )
    return conn

In [3]:
connect_to_db().is_connected()

True

### Defining an agnet "cursor" who will communicate with db

In [4]:
db = connect_to_db()
cursor = db.cursor(dictionary=True)

In [6]:
cursor.execute("select count(*) as cnt from products") #this is just execute query on MySQL but will not return anything here

In [7]:
row = cursor.fetchone()

In [8]:
row

{'cnt': 201}

In [9]:
list(row.values())[0]

201

In [10]:
queries = {
    "Total Suppliers": "select count(*) as total_suppliers from suppliers;",

    "Total Products": "select count(*) as total_products from products;",

    "Total Categories Dealing": "select count(distinct category) as total_categories from products;",

    "Total Sale Value (last 3 months)": """ 
        select round(sum(abs(se.change_quantity) * p.price), 2) as total_sales_value_in_last_3_months
        from stock_entries as se
        join products as p 
        on p.product_id = se.product_id
        where se.change_type='Sale' and se.entry_date >= (select date_sub(max(entry_date), interval 3 month) from stock_entries);
        """,

    "Total Restock Value (last 3 months)": """ 
        select round(sum(abs(se.change_quantity) * p.price), 2) as total_restock_value_in_last_3_months
        from stock_entries as se
        join products as p 
        on p.product_id = se.product_id
        where se.change_type='Restock' and se.entry_date >= (select date_sub(max(entry_date), interval 3 month) from stock_entries);
        """,

    "Below Reorders and No Pending Reorders": """ 
        select count(*) from products as p where p.stock_quantity < p.reorder_level
        and product_id not in (
        select distinct product_id from reorders where status='Pending'
        )
        """
}

In [11]:
result={}

for label, query in queries.items():
    cursor.execute(query)
    row = cursor.fetchone()
    result[label] = list(row.values())[0]

In [12]:
result

{'Total Suppliers': 50,
 'Total Products': 201,
 'Total Categories Dealing': 5,
 'Total Sale Value (last 3 months)': None,
 'Total Restock Value (last 3 months)': 9999.0,
 'Below Reorders and No Pending Reorders': 14}

In [13]:
def get_basic_info(cursor):
    queries = {
    "Total Suppliers": "select count(*) as total_suppliers from suppliers;",

    "Total Products": "select count(*) as total_products from products;",

    "Total Categories Dealing": "select count(distinct category) as total_categories from products;",

    "Total Sale Value (last 3 months)": """ 
        select round(sum(abs(se.change_quantity) * p.price), 2) as total_sales_value_in_last_3_months
        from stock_entries as se
        join products as p 
        on p.product_id = se.product_id
        where se.change_type='Sale' and se.entry_date >= (select date_sub(max(entry_date), interval 3 month) from stock_entries);
        """,

    "Total Restock Value (last 3 months)": """ 
        select round(sum(abs(se.change_quantity) * p.price), 2) as total_restock_value_in_last_3_months
        from stock_entries as se
        join products as p 
        on p.product_id = se.product_id
        where se.change_type='Restock' and se.entry_date >= (select date_sub(max(entry_date), interval 3 month) from stock_entries);
        """,

    "Below Reorders and No Pending Reorders": """ 
        select count(*) from products as p where p.stock_quantity < p.reorder_level
        and product_id not in (
        select distinct product_id from reorders where status='Pending'
        )
        """
    }

    result={}

    for label, query in queries.items():
        cursor.execute(query)
        row = cursor.fetchone()
        result[label] = list(row.values())[0]
    
    return result

In [14]:
queries = {
    "Suppliers contact details": "select supplier_name, contact_name, email, phone from suppliers;",

    "Product with suppliers and stock": """
    select p.product_name, s.supplier_name, p.stock_quantity, p.reorder_level
    from products as p
    join suppliers as s
    on p.supplier_id = s.supplier_id
    order by p.product_name;
    """,

    "Product needing reorder":""" 
    select product_id, product_name, stock_quantity, reorder_level
    from products 
    where stock_quantity < reorder_level
    """
}

In [15]:
table={}

for label, query in queries.items():
    cursor.execute(query)
    table[label] = cursor.fetchall()

In [16]:
table

{'Suppliers contact details': [{'supplier_name': 'Anderson-Thompson',
   'contact_name': 'Bonnie Davis',
   'email': 'zacharysanchez@hotmail.com',
   'phone': '829.485.9853x0522'},
  {'supplier_name': 'Rowland Ltd',
   'contact_name': 'Beth Stevens',
   'email': 'joshua60@yahoo.com',
   'phone': '(779)942-0726'},
  {'supplier_name': 'Baxter-Meadows',
   'contact_name': 'Lisa Lewis',
   'email': 'andersonchristina@yahoo.com',
   'phone': '449-766-7325'},
  {'supplier_name': 'Wilson, Graham and Williams',
   'contact_name': 'David Martinez',
   'email': 'abrown@hotmail.com',
   'phone': '+1-653-827-5215x266'},
  {'supplier_name': 'Smith, Kennedy and Moreno',
   'contact_name': 'Victoria Gonzalez',
   'email': 'mary30@williams-moore.com',
   'phone': '891-859-2775x35297'},
  {'supplier_name': 'Middleton LLC',
   'contact_name': 'Megan Miller',
   'email': 'kelseywilliams@gmail.com',
   'phone': '001-724-731-6199x4596'},
  {'supplier_name': 'Evans Inc',
   'contact_name': 'Danielle Moore',

In [17]:
def get_additional_tables(cursor):
    queries = {
    "Suppliers contact details": "select supplier_name, contact_name, email, phone from suppliers;",

    "Product with suppliers and stock": """
    select p.product_name, s.supplier_name, p.stock_quantity, p.reorder_level
    from products as p
    join suppliers as s
    on p.supplier_id = s.supplier_id
    order by p.product_name;
    """,

    "Product needing reorder":""" 
    select product_id, product_name, stock_quantity, reorder_level
    from products 
    where stock_quantity < reorder_level
    """
    }
    
    tables={}

    for label, query in queries.items():
        cursor.execute(query)
        tables[label] = cursor.fetchall()

    return tables

In [18]:
get_additional_tables(cursor)

{'Suppliers contact details': [{'supplier_name': 'Anderson-Thompson',
   'contact_name': 'Bonnie Davis',
   'email': 'zacharysanchez@hotmail.com',
   'phone': '829.485.9853x0522'},
  {'supplier_name': 'Rowland Ltd',
   'contact_name': 'Beth Stevens',
   'email': 'joshua60@yahoo.com',
   'phone': '(779)942-0726'},
  {'supplier_name': 'Baxter-Meadows',
   'contact_name': 'Lisa Lewis',
   'email': 'andersonchristina@yahoo.com',
   'phone': '449-766-7325'},
  {'supplier_name': 'Wilson, Graham and Williams',
   'contact_name': 'David Martinez',
   'email': 'abrown@hotmail.com',
   'phone': '+1-653-827-5215x266'},
  {'supplier_name': 'Smith, Kennedy and Moreno',
   'contact_name': 'Victoria Gonzalez',
   'email': 'mary30@williams-moore.com',
   'phone': '891-859-2775x35297'},
  {'supplier_name': 'Middleton LLC',
   'contact_name': 'Megan Miller',
   'email': 'kelseywilliams@gmail.com',
   'phone': '001-724-731-6199x4596'},
  {'supplier_name': 'Evans Inc',
   'contact_name': 'Danielle Moore',

In [19]:
def add_new_manual_id(cursor, db, p_name, p_category, p_price, p_stock, p_reorder, p_supplier):
    proc_call = "call AddNewProductManualID(%s, %s, %s, %s, %s, %s)"
    params = (p_name, p_category, p_price, p_stock, p_reorder, p_supplier)
    cursor.execute(proc_call, params)
    db.commit()

In [26]:
def get_categories(cursor):
    cursor.execute("select distinct category from products order by category asc")
    rows = cursor.fetchall()
    return [row['category'] for row in rows]

In [27]:
get_categories(cursor)

['Clothing', 'Electronics', 'Furniture', 'Groceries', 'Toys']

In [28]:
def get_suppliers(cursor):
    cursor.execute("select supplier_id, supplier_name from suppliers order by supplier_name asc")
    return cursor.fetchall()

In [30]:
suppliers = get_suppliers(cursor)

In [ ]:
supplier_ids = [s['supplier_id'] for s in suppliers]
supplier_names = [s['supplier_name'] for s in suppliers]

f = lambda x: supplier_names[supplier_ids.index(x)]

In [38]:
result = [f(x) for x in supplier_ids]
print(result)


['Anderson-Thompson', 'Armstrong-Vance', 'Barker, White and Carson', 'Barrett Ltd', 'Baxter-Meadows', 'Charles Inc', 'Clark Group', 'Douglas Ltd', 'Elliott-Ayers', 'Evans Inc', 'Franklin, Kane and Price', 'Freeman-Gordon', 'Gallagher-Miller', 'Gomez PLC', 'Hall-Brown', 'Harris-Cummings', 'Henderson LLC', 'Hensley-Branch', 'Hudson Inc', 'Johnson-Bass', 'Kaufman Ltd', 'Lawrence, Garcia and Hernandez', 'Lloyd and Sons', 'Mann-Marshall', 'Mendoza-Jones', 'Middleton LLC', 'Miller-Martinez', 'Moody-Vang', 'Morgan Inc', 'Morgan-Andrews', 'Moss-Evans', 'Newton, Valencia and Carr', 'Ortega-Mahoney', 'Patrick, Walter and Harrison', 'Perez, Price and Wallace', 'Reynolds-Phillips', 'Rogers-Greene', 'Rowe PLC', 'Rowland Ltd', 'Smith, Kennedy and Moreno', 'Stewart, Williams and Cox', 'Taylor-Love', 'Tran LLC', 'Tucker-Arnold', 'Turner-Davis', 'Vega, Cook and Miller', 'Williams Ltd', 'Wilson, Graham and Williams', 'Wong Group', 'Young, Browning and Ware']
